In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import sys
sys.path.insert(0, '/content/drive/My Drive/VAE/')
sys.path.append('/content/drive/My Drive/VAE')
sys.path.append('/content/drive/Cancer-Detection/data/train/benign/')

In [0]:
! git clone --recursive https://github.com/shreyavarshini/Cancer-Detection.git

Cloning into 'Cancer-Detection'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 18717 (delta 4), reused 26 (delta 2), pack-reused 18689
Receiving objects: 100% (18717/18717), 318.63 MiB | 12.90 MiB/s, done.
Resolving deltas: 100% (4/4), done.
Checking out files: 100% (18679/18679), done.


In [0]:
# imports and initializations
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms, transforms, utils
from torchvision.utils import save_image
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from torch.autograd import Variable

In [0]:
# Arguments for the model
class Args:
  batch_size = 20
  epochs = 60
  no_cuda = False
  seed = 1
  log_interval = 10
  lr = 1e-4

args=Args()
args.cuda = not args.no_cuda and torch.cuda.is_available()
torch.manual_seed(args.seed)
device = torch.device("cuda" if args.cuda else "cpu")

In [0]:
# Dataset loader
def load_dataset( folder ):
    data_path = '/content/Cancer-Detection/data/' + folder
    transform = transforms.Compose([
    transforms.Resize(( 128, 128 ) ),
    transforms.ToTensor()
    ])
    dataset = datasets.ImageFolder( root=data_path, transform=transform )

    loader = torch.utils.data.DataLoader(
        dataset,
        batch_size=1,
        num_workers=5,
        shuffle=True
    )
    return loader

train_loader1 = load_dataset( 'train/benign_train' )
train_loader = load_dataset( 'train/malignant_train' )

test_loader1 = load_dataset( 'test/benign_test' )
test_loader = load_dataset( 'test/malignant_test' )


In [0]:
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, z_dim):
        '''
        Args:
            input_dim: A integer indicating the size of input
            hidden_dim: A integer indicating the size of hidden dimension
            z_dim: A integer indicating the latent dimension
        '''
        super().__init__()

        self.linear = nn.Linear(input_dim, hidden_dim)
        self.mu = nn.Linear(hidden_dim, z_dim)
        self.var = nn.Linear(hidden_dim, z_dim)

    def forward(self, x):
        # x is of shape [batch_size, input_dim]

        hidden = F.relu(self.linear(x))
        # hidden is of shape [batch_size, hidden_dim]
        z_mu = self.mu(hidden)
        # z_mu is of shape [batch_size, latent_dim]
        z_var = self.var(hidden)
        # z_var is of shape [batch_size, latent_dim]

        return z_mu, z_var

In [0]:
class Decoder(nn.Module):
    def __init__(self, z_dim, hidden_dim, output_dim):
        '''
        Args:
            z_dim: A integer indicating the latent size.
            hidden_dim: A integer indicating the size of hidden dimension.
            output_dim: A integer indicating the output dimension
        '''
        super().__init__()

        self.linear = nn.Linear(z_dim, hidden_dim)
        self.out = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # x is of shape [batch_size, latent_dim]

        hidden = F.relu(self.linear(x))
        # hidden is of shape [batch_size, hidden_dim]

        predicted = torch.sigmoid(self.out(hidden))
        # predicted is of shape [batch_size, output_dim]
        return predicted


In [0]:
class VAE(nn.Module):
    def __init__(self, enc, dec):
        ''' This the VAE, which takes a encoder and decoder.
        '''
        super().__init__()

        self.encode = enc
        self.decode = dec

    def forward(self, x):
        # encode
        z_mu, z_var = self.encode(x)

        # sample from the distribution having latent parameters z_mu, z_var
        # reparameterize
        std = torch.exp(z_var / 2)
        eps = torch.randn_like(std)
        x_sample = eps.mul(std).add_(z_mu)

        # decode
        predicted = self.decode(x_sample)
        return predicted, z_mu, z_var

In [0]:
INPUT_DIM = 49152
HIDDEN_DIM = 400
LATENT_DIM = 50

# encoder
encoder = Encoder(INPUT_DIM, HIDDEN_DIM, LATENT_DIM)

# decoder
decoder = Decoder(LATENT_DIM, HIDDEN_DIM, INPUT_DIM)

# vae
model = VAE(encoder, decoder).to(device)

# optimizer
optimizer = optim.Adam(model.parameters(), lr=args.lr)

In [0]:
from google.colab import files

def loss_function(recon_x, x, mu, logvar):
    # how well do input x and output recon_x agree?
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 49152))

    # KLD is Kullback–Leibler divergence -- how much does one learned
    # distribution deviate from another, in this specific case the
    # learned distribution from the unit Gaussian

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # - D_{KL} = 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    # note the negative D_{KL} in appendix B of the paper
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    # Normalise by same number of elements as in reconstruction
    KLD /= 20 * 49152

    # BCE tries to make our reconstruction as accurate as possible
    # KLD tries to push the distributions as close as possible to unit Gaussian
    return BCE + KLD

def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.view(-1, 3 * 128 * 128)
        data = data.to(device)

        # update the gradients to zero
        optimizer.zero_grad()

        # forward pass
        recon_batch, mu, logvar = model(data)


        # reconstruction loss + kl divergence loss
        loss = loss_function(recon_batch, data, mu, logvar)

        
         # backward pass
        loss.backward()
        train_loss += loss.item()

        # update the weights
        optimizer.step()

        if batch_idx % args.log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))


def test(epoch):
    # set the evaluation mode
    model.eval()

    # test loss for the data
    test_loss = 0

    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = data.view(-1, 3 * 128 * 128)
            data = data.to(device)

            # forward pass
            recon_batch, mu, logvar = model(data)

            # total loss
            test_loss += loss_function(recon_batch, data, mu, logvar).item()

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

if __name__ == "__main__":
    for epoch in range(1, args.epochs + 1):
        train(epoch)
        test(epoch)
        with torch.no_grad():
            sample = torch.randn(1, LATENT_DIM).to(device)
            sample = model.decode(sample).cpu()
            img = sample.view(3, 128, 128)
            img = img.permute(1, 2, 0)
            plt.figure()
            plt.imshow(img)
            plt.show()
    with torch.no_grad():
      for i in range(1000):
        sample = torch.randn(1, LATENT_DIM).to(device)
        sample = model.decode(sample).cpu()
        img = sample.view(3, 128, 128)
        img = img.permute(1, 2, 0)
        
        plt.figure()
        _ = plt.imshow(img)
        name = '/content/Cancer-Detection/Generated_data/Malignant/image_' + str(i) + ".png" 
        plt.savefig(name)
    
    ! zip -r "/content/images_Malignant.zip" "/content/Cancer-Detection/Generated_data/Malignant/"

In [14]:
!ps -aux|grep python

root          19  0.5  0.8 413692 110264 ?       Sl   14:23   1:06 /usr/bin/python2 /usr/local/bin/jupyter-notebook --ip="172.28.0.2" --port=9000 --FileContentsManager.root_dir="/" --MappingKernelManager.root_dir="/content"
root       11775  8.1 19.8 18505540 2644516 ?    Ssl  17:34   0:20 /usr/bin/python3 -m ipykernel_launcher -f /root/.local/share/jupyter/runtime/kernel-c9f09194-5edc-4a90-aa53-771fedb1101b.json
root       11915  0.0  0.0  39192  6616 ?        S    17:38   0:00 /bin/bash -c ps -aux|grep python
root       11917  0.0  0.0  39192  2700 ?        D    17:38   0:00 /bin/bash -c ps -aux|grep python


In [0]:
! kill -9 11775